In [124]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN

In [103]:
people

,name,roles,profession,processed date,Archive
4569,...-a,NaN,nubanda[overseer],67.0505,domesticated animal
22306,...-a,NaN,udu[sheep],62.0400,domesticated animal
22613,...-a,NaN,NaN,61.0401,dead animal
22617,...-a,NaN,NaN,75.0315,domesticated animal
10372,...-a,NaN,NaN,66.0612,dead animal
22611,...-a,NaN,NaN,NaN,queen
22292,...-a,NaN,udu[sheep],NaN,domesticated animal
3177,...-a,NaN,NaN,85.1000,domesticated animal
15336,...-a,NaN,NaN,71.0100,dead animal
22604,...-a,{'source'},NaN,NaN,queen


In [100]:
people = pd.read_csv("people_Drehem.csv")
archives = pd.read_csv("archive_map.csv")
archives['PID'] = [int(archives['PID'][i][1:]) for i in np.arange(len(archives['PID']))]
people = pd.merge(people, archives, how='inner', left_on='p index', right_on='PID')
people = people.drop(['PID', 'normalized name', 'family', 'date name', 'p index'], axis=1)
people = people.sort_values(['name'])

Our people dataframe contains columns for the variables we want to use on our clustering algorithm. Our next step is to convert our categorical data into numerical data. We can do this by creating a binary matrix. 

In [87]:
people_cat = people.select_dtypes(include=[object])
people_cat['profession'] = people_cat['profession'].astype(str)
people_cat['roles'] = people_cat['roles'].astype(str)
people_cat.head(3)

/Users/anjaliunnithan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/anjaliunnithan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,name,roles,profession,Archive
4569,...-a,nan,nubanda[overseer],domesticated animal
22306,...-a,nan,udu[sheep],domesticated animal
22613,...-a,nan,nan,dead animal


In [92]:
# import preprocessing from sklearn
from sklearn import preprocessing

In [93]:
people_cat.columns

Index(['name', 'roles', 'profession', 'Archive'], dtype='object')

In [96]:
# TODO: create a LabelEncoder object and fit it to each feature in people_cat


# 1. INSTANTIATE
# encode labels with value between 0 and n_classes-1.
le = preprocessing.LabelEncoder()


# 2/3. FIT AND TRANSFORM
# use df.apply() to apply le.fit_transform to all columns
people_cat_2 = people_cat.apply(le.fit_transform)
people_cat_2

,name,roles,profession,Archive
4569,0,0,153,1
22306,0,0,195,1
22613,0,0,145,0
22617,0,0,145,1
10372,0,0,145,0
22611,0,0,145,4
22292,0,0,195,1
3177,0,0,145,1
15336,0,0,145,0
22604,0,14,145,4


In [97]:
# TODO: create a OneHotEncoder object, and fit it to all of people_cat

# 1. INSTANTIATE
enc = preprocessing.OneHotEncoder()

# 2. FIT
enc.fit(people_cat_2)

# 3. Transform
onehotlabels = enc.transform(people_cat_2).toarray()
onehotlabels.shape

# as you can see, you've the same number of rows 891
# but now you've so many more columns due to how we changed all the categorical data into numerical data

(43017, 4415)

In [98]:
onehotlabels

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

We just converted our categorical data into a sparse matrix. We should add this matrix to our quantitative data. 

In [122]:
dates = [[people['processed date'].values[i]] for i in np.arange(len(people['processed date']))]
people_cat_quant = np.hstack((onehotlabels, dates))
people_cat_quant

array([[ 1.    ,  0.    ,  0.    , ...,  0.    ,  0.    , 67.0505],
       [ 1.    ,  0.    ,  0.    , ...,  0.    ,  0.    , 62.04  ],
       [ 1.    ,  0.    ,  0.    , ...,  0.    ,  0.    , 61.0401],
       ...,
       [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    , 74.0905],
       [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    , 50.1   ],
       [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    , 68.08  ]])

Our matrix is now ready to be passed through the DBSCAN algorithm. 